In [1]:
import os
import glob
import pandas as pd
import matplotlib.pyplot as plt
import pickle 
import cv2
import mediapipe as mp
import numpy as np
from time import time
from datetime import datetime, timedelta

# Mediapipe configurations
mp_face_detection = mp.solutions.face_detection
mp_face_mesh = mp.solutions.face_mesh
mp_drawing = mp.solutions.drawing_utils

# Timestamp 데이터 프레임 정리 

In [2]:
Video = '/Users/iduli/Desktop/Ch2_25_Scientific_Data/Final/raw_Dataset/SENSORS/VIDEO'
raw_video_dir = os.path.join(Video, "raw_video")
video_dir = os.path.join(Video)
output_dir = os.path.join(Video, "results")
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# 1st 폴더 처리
folder_1st = os.path.join(raw_video_dir, "1st")
time_csv_1st = os.path.join(video_dir, "1st_time.csv")

# 2nd 폴더 처리
folder_2nd = os.path.join(raw_video_dir, "2nd")
time_csv_2nd = os.path.join(video_dir, "2nd_time.csv")

In [3]:
def get_baseline_time_for_condition(subject_num, condition, raw_audio_folder):
    filename = f"{subject_num}_1_{condition}_webcam-logger.csv"
    file_path = os.path.join(raw_audio_folder, filename)

    if not os.path.exists(file_path):
        return None
    df = pd.read_csv(file_path)
    # 첫 번째 행의 global_time을 baseline으로 사용
    baseline_time = df.loc[0, "global_time"]
    #print(baseline_time)
    return baseline_time

def parse_mmssms_to_timedelta(time_str):

    if pd.isnull(time_str) or time_str.strip() == "":
        return None
    mm, ss, ms_str = time_str.split(";")
    mm = int(mm)
    ss = int(ss)
    # '미리초(0~99)' * 10 => 실제 milliseconds
    ms = int(ms_str) * 10
    total_milliseconds = mm * 60000 + ss * 1000 + ms
    #print(total_milliseconds)
    return total_milliseconds

In [4]:
# 2nd 
time_df = pd.read_csv(time_csv_2nd)
results = []
for _, row in time_df.iterrows():
    pnum = row["pnum"]
    condition = str(row["condition"]).strip()
    #print(pnum, condition)
    c1_str = row["c1"]
    c2_str = row["c2"]
    c3_str = row["c3"]
    baseline = get_baseline_time_for_condition(pnum, condition, folder_2nd)
    if baseline is None:
        # 해당 파일이 없으면 None으로 두거나, 로그를 남길 수 있음
        c1_utc = None
        c2_utc = None
        c3_utc = None
    else:
        # 2) c1, c2, c3 각각 파싱
        
        c1_offset = parse_mmssms_to_timedelta(c1_str)
        c2_offset = parse_mmssms_to_timedelta(c2_str)
        c3_offset = parse_mmssms_to_timedelta(c3_str)

        c1_utc = int(baseline) + int(c1_offset) if c1_offset else None
        c2_utc = int(baseline) + int(c2_offset) if c2_offset else None
        c3_utc = int(baseline) + int(c3_offset) if c3_offset else None
    results.append({
        "pnum": pnum,
        "c1": c1_utc,
        "c2": c2_utc,
        "c3": c3_utc
    })

pd.set_option('display.float_format', lambda x: f'{x:.0f}')

time_2 = pd.DataFrame(results)
# pnum, condition 순으로 정렬
time_2.sort_values(by=["pnum"], inplace=True)
time_2_grouped = time_2.groupby("pnum", as_index=False).first()
print(time_2_grouped)

    pnum            c1            c2            c3
0      1 1708133874749 1708134415179 1708135064019
1      2 1708138921695 1708139374615 1708139837415
2      4 1708149174093 1708149673917 1708150138067
3      5 1708154998132 1708155464002 1708156023392
4      6 1708160855018 1708161215418 1708161682458
5      7 1708220866360 1708221284930 1708221830720
6      8 1708224638019 1708225072039 1708225517089
7     10 1708236380941 1708236820081 1708237334871
8     11 1708241795392 1708242291402 1708242944622
9     12 1708307665621 1708308079571 1708308581531
10    13 1708311237274 1708312013614 1708312435544
11    14 1708316103451 1708316560461 1708317021341
12    15 1708322961823 1708323444973 1708323913713


In [5]:
# 1st 
time_df = pd.read_csv(time_csv_1st)
results = []
for _, row in time_df.iterrows():
    pnum = row["pnum"]
    condition = str(row["condition"]).strip()
    #print(pnum, condition)
    c1_str = row["c1"]
    c2_str = row["c2"]
    c3_str = row["c3"]
    baseline = get_baseline_time_for_condition(pnum, condition, folder_1st)
    if baseline is None:
        # 해당 파일이 없으면 None으로 두거나, 로그를 남길 수 있음
        c1_utc = None
        c2_utc = None
        c3_utc = None
    else:
        # 2) c1, c2, c3 각각 파싱
        
        c1_offset = parse_mmssms_to_timedelta(c1_str)
        c2_offset = parse_mmssms_to_timedelta(c2_str)
        c3_offset = parse_mmssms_to_timedelta(c3_str)

        c1_utc = int(baseline) + int(c1_offset) if c1_offset else None
        c2_utc = int(baseline) + int(c2_offset) if c2_offset else None
        c3_utc = int(baseline) + int(c3_offset) if c3_offset else None
    results.append({
        "pnum": pnum,
        "c1": c1_utc,
        "c2": c2_utc,
        "c3": c3_utc
    })

pd.set_option('display.float_format', lambda x: f'{x:.0f}')

time_1 = pd.DataFrame(results)
# pnum, condition 순으로 정렬
time_1.sort_values(by=["pnum"], inplace=True)
time_1_grouped = time_1.groupby("pnum", as_index=False).first()

print(time_1_grouped)

    pnum            c1            c2            c3
0      1 1671068336785 1671068806645 1671069238335
1      2 1671072379186 1671072795326 1671073226566
2      3 1671078713202 1671080082890 1671079535240
3      4 1671083392936 1671083821136 1671084427321
4      8 1671167300402 1671167843772 1671168355472
5     10 1671176153254 1671176527614 1671176950044
6     12 1671245406272 1671245867362 1671246283292
7     13 1671251228083 1671251839213 1671252301653
8     14 1671256497984 1671256889694 1671257299414
9     16 1671267813860 1671268230560 1671268630930
10    18 1671332110114 1671332508724 1671332923974
11    19 1671337390010 1671337759720 1671338152320
12    20 1671343176119 1671343685068 1671344101538
13    21 1671348410357 1671348821827 1671349396847
14    22 1671353003060 1671353409090 1671353984170
15    23 1671429960227 1671430379247 1671430797957
16    25 1671587643730 1671588031700 1671588539130
17    26 1671672896380 1671673328260 1671673828350
18    27 1671688733167 16716891

# feature extraction

* 타임스탬프 값 더해서 추출 하기 !!! 

In [6]:
# Mediapipe, OpenCV and interval configurations
EAR_THRESHOLD = 0.3
FOURCC = 'XVID'

In [7]:
def save_features_to_csv(df_features, file_path):
    df_features.to_csv(file_path, index=False)

# Calculate the eye aspect ratio
def calculate_eye_aspect_ratio(eye_landmarks):
    A = np.linalg.norm(eye_landmarks[1] - eye_landmarks[5])
    B = np.linalg.norm(eye_landmarks[2] - eye_landmarks[4])
    C = np.linalg.norm(eye_landmarks[0] - eye_landmarks[3])
    return (A + B) / (2.0 * C)

# Calculate the distance between two points
def normalize_vector(v):
    norm = np.linalg.norm(v)
    if norm == 0:
        return v
    return v / norm


In [8]:
def process_video(video_path):
    frame_idx = 0
    frame_time = 0
    # Open video
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Cannot open video {video_path}")
        return None    
    folder_path, video_file = os.path.split(video_path)
    print(video_file)
    base_name = os.path.splitext(video_file)[0]
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    # Initialize variables
    feature_list = []
    prev_lip_corner_left, prev_lip_corner_right = None, None
    prev_left_eye_aspect_ratio, prev_right_eye_aspect_ratio = None, None
    blink_count = 0

    with mp_face_detection.FaceDetection(model_selection=1, min_detection_confidence=0.7) as face_detection, \
         mp_face_mesh.FaceMesh(min_detection_confidence=0.7, min_tracking_confidence=0.7, refine_landmarks= True) as face_mesh:
        #print(frame_idx)
        while cap.isOpened():
            # Set frame time, will be added after 영상 시작시간  
            #current_frame_idx = int(cap.get(cv2.CAP_PROP_POS_FRAMES))
            #print(current_frame_idx) frame 0 1 같은시간 읽어짐. 
            frame_time = cap.get(cv2.CAP_PROP_POS_MSEC)
            success, image = cap.read()
            error_type = 0
            if not success:
                # Case 1: 프레임 읽기 실패
                error_type = 1
                frame_features = [frame_time, np.nan, np.nan, np.nan, np.nan,
                                  np.nan, np.nan, np.nan, np.nan, np.nan, error_type]  
                break

            # Convert BGR to RGB for Mediapipe
            image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            image_rgb.flags.writeable = False

            # Process face detection and face mesh
            results = face_detection.process(image_rgb)
            mesh_results = face_mesh.process(image_rgb)

            if not results.detections:
                # Case 2: 얼굴 탐지 실패
                error_type = 2
                frame_features = [frame_time, np.nan, np.nan, np.nan, np.nan,
                                  np.nan, np.nan, np.nan, np.nan, np.nan, error_type]  
                feature_list.append(frame_features)
                continue

            # Convert RGB back to BGR for OpenCV
            image_rgb.flags.writeable = True
            image = cv2.cvtColor(image_rgb, cv2.COLOR_RGB2BGR)

            # Initialize default values
            rotation_x, rotation_y, rotation_z = np.nan, np.nan, np.nan
            total_translation = np.nan
            gaze_direction = np.array([np.nan, np.nan])
            left_lip_movement, right_lip_movement = np.nan, np.nan

            # Process face detection
            if results.detections:
                largest_face = max(results.detections, key=lambda detection: detection.location_data.relative_bounding_box.width * detection.location_data.relative_bounding_box.height)
                blink_count = 0  # Reset blink count for each frame

            # Process face mesh landmarks
            if mesh_results.multi_face_landmarks:
                for face_landmarks in mesh_results.multi_face_landmarks:
                    landmarks = np.array([(lm.x, lm.y, lm.z) for lm in face_landmarks.landmark])

                    # Calculate head movement
                    image_points = np.array([
                        (landmarks[1][0] * image.shape[1], landmarks[1][1] * image.shape[0]),
                        (landmarks[33][0] * image.shape[1], landmarks[33][1] * image.shape[0]),
                        (landmarks[263][0] * image.shape[1], landmarks[263][1] * image.shape[0]),
                        (landmarks[61][0] * image.shape[1], landmarks[61][1] * image.shape[0]),
                        (landmarks[291][0] * image.shape[1], landmarks[291][1] * image.shape[0]),
                        (landmarks[199][0] * image.shape[1], landmarks[199][1] * image.shape[0])
                    ], dtype="double")

                    '''considering changing the points from static value to dynamic value based on landmarks'''
                    model_points = np.array([
                        (0.0, 0.0, 0.0),
                        (-30.0, -125.0, -30.0),
                        (30.0, -125.0, -30.0),
                        (-60.0, -70.0, -60.0),
                        (60.0, -70.0, -60.0),
                        (0.0, -150.0, -100.0)
                    ]) 
                    '''model_points = np.array([
                                    (landmarks[1][0], landmarks[1][1], landmarks[1][2]),    # 코 끝
                                    (landmarks[33][0], landmarks[33][1], landmarks[33][2]), # 왼쪽 눈
                                    (landmarks[263][0], landmarks[263][1], landmarks[263][2]), # 오른쪽 눈
                                    (landmarks[61][0], landmarks[61][1], landmarks[61][2]), # 왼쪽 입술
                                    (landmarks[291][0], landmarks[291][1], landmarks[291][2]), # 오른쪽 입술
                                    (landmarks[199][0], landmarks[199][1], landmarks[199][2])  # 턱 끝
                                ])'''
                    size = image.shape
                    focal_length = size[1]
                    #print(focal_length)
                    center = (size[1] / 2, size[0] / 2)
                    camera_matrix = np.array([
                        [focal_length, 0, center[0]],
                        [0, focal_length, center[1]],
                        [0, 0, 1]
                    ], dtype="double")
                    #print(center[0])

                    dist_coeffs = np.zeros((4, 1))
                    success, rotation_vector, translation_vector = cv2.solvePnP(model_points, image_points, camera_matrix, dist_coeffs)

                    if success:
                        rotation_x, rotation_y, rotation_z = rotation_vector.ravel()
                        total_translation = np.linalg.norm(translation_vector)
                        #print('head')

                        # Calculate eye aspect ratio
                        left_eye_landmarks = landmarks[[33, 160, 158, 133, 153, 144]]
                        right_eye_landmarks = landmarks[[362, 385, 387, 263, 373, 380]]
                        left_ear = calculate_eye_aspect_ratio(left_eye_landmarks)
                        right_ear = calculate_eye_aspect_ratio(right_eye_landmarks)

                        if prev_left_eye_aspect_ratio is not None and prev_right_eye_aspect_ratio is not None:
                            if (left_ear < EAR_THRESHOLD and prev_left_eye_aspect_ratio >= EAR_THRESHOLD) or \
                               (right_ear < EAR_THRESHOLD and prev_right_eye_aspect_ratio >= EAR_THRESHOLD):
                                blink_count += 1
                        #print('eye')

                        prev_left_eye_aspect_ratio = left_ear
                        prev_right_eye_aspect_ratio = right_ear

                        # Calculate gaze direction
                        
                        left_iris_center = np.mean(landmarks[[474, 475, 476, 477]], axis=0)
                        right_iris_center = np.mean(landmarks[[469, 470, 471, 472]], axis=0)
                        nose_tip = landmarks[1]
                        gaze_direction = normalize_vector((left_iris_center + right_iris_center) / 2.0 - nose_tip)
                        #print('gaze')
                        # Calculate lip movement
                        left_lip_corner = landmarks[61]
                        right_lip_corner = landmarks[291]
                        if prev_lip_corner_left is not None and prev_lip_corner_right is not None:
                            left_lip_movement = np.linalg.norm(left_lip_corner - nose_tip) - np.linalg.norm(prev_lip_corner_left - nose_tip)
                            right_lip_movement = np.linalg.norm(right_lip_corner - nose_tip) - np.linalg.norm(prev_lip_corner_right - nose_tip)
                        #print('lip')
                        prev_lip_corner_left = left_lip_corner
                        prev_lip_corner_right = right_lip_corner
            # Append features
            frame_features = [frame_time, rotation_x, rotation_y, rotation_z, total_translation,
                              gaze_direction[0], gaze_direction[1], left_lip_movement, right_lip_movement, blink_count, error_type]

            feature_list.append(frame_features)
            frame_idx += 1

    cap.release()
    csv_file_path = os.path.join(result_folder, f'{base_name}.csv')
    save_features_to_csv(pd.DataFrame(feature_list, columns=[
        "Timestamp", "rotation_x", "rotation_y", "rotation_z",
        "Total Movement", "Gaze X", "Gaze Y",
        "Left Lip Movement", "Right Lip Movement",
        "Blink Count",'error_type']), csv_file_path)

In [9]:
video_folder = '/Users/iduli/Desktop/Ch2_25_Scientific_Data/Final/raw_Dataset/SENSORS/VIDEO/parsed_video/1st'
result_folder = '/Users/iduli/Desktop/Ch2_25_Scientific_Data/Final/raw_Dataset/SENSORS/VIDEO/features/1st'


for file in os.listdir(video_folder): 
    process_video(os.path.join(video_folder,file))

20_c3.mp4


I0000 00:00:1744334962.228320 7797518 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 88.1), renderer: Apple M1
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
I0000 00:00:1744334962.234631 7797518 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 88.1), renderer: Apple M1
W0000 00:00:1744334962.236934 7798146 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1744334962.245063 7798150 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1744334962.247275 7798128 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1744334962.255714 7798148 landmark_projection_calculator.cc:186] Using NORM_RECT without IMAGE_DIMENSIONS is only supported for the square ROI. Provide IMAGE_DIME

16_c2.mp4


I0000 00:00:1744335023.308208 7797518 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 88.1), renderer: Apple M1
I0000 00:00:1744335023.314518 7797518 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 88.1), renderer: Apple M1
W0000 00:00:1744335023.317500 7798859 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1744335023.325870 7798861 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1744335023.327823 7798855 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


16_c3.mp4


...

I0000 00:00:1744338285.180251 7797518 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 88.1), renderer: Apple M1
I0000 00:00:1744338285.186690 7797518 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 88.1), renderer: Apple M1
W0000 00:00:1744338285.188989 7835871 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1744338285.196492 7835875 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1744338285.198845 7835864 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


25_c2.mp4


I0000 00:00:1744338345.360810 7797518 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 88.1), renderer: Apple M1
I0000 00:00:1744338345.366021 7797518 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 88.1), renderer: Apple M1
W0000 00:00:1744338345.368318 7836393 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1744338345.377888 7836393 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1744338345.378915 7836380 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


13_c3.mp4


I0000 00:00:1744338404.203710 7797518 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 88.1), renderer: Apple M1
I0000 00:00:1744338404.209368 7797518 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 88.1), renderer: Apple M1
W0000 00:00:1744338404.211647 7837041 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1744338404.218345 7837040 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1744338404.222221 7837034 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


In [10]:
video_folder = '/Users/iduli/Desktop/Ch2_25_Scientific_Data/Final/raw_Dataset/SENSORS/VIDEO/parsed_video/2nd'
result_folder = '/Users/iduli/Desktop/Ch2_25_Scientific_Data/Final/raw_Dataset/SENSORS/VIDEO/features/2nd'


for file in os.listdir(video_folder): 
    process_video(os.path.join(video_folder,file))

3_c1.mp4


I0000 00:00:1744338462.503780 7797518 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 88.1), renderer: Apple M1
I0000 00:00:1744338462.509961 7797518 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 88.1), renderer: Apple M1
W0000 00:00:1744338462.512016 7837491 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1744338462.520206 7837495 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1744338462.523231 7837479 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


3_c3.mp4


I0000 00:00:1744338521.935130 7797518 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 88.1), renderer: Apple M1
I0000 00:00:1744338521.941491 7797518 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 88.1), renderer: Apple M1
W0000 00:00:1744338521.943076 7837995 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1744338521.949378 7837991 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1744338521.952766 7837984 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


3_c2.mp4


...

I0000 00:00:1744340739.078774 7797518 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 88.1), renderer: Apple M1
I0000 00:00:1744340739.085576 7797518 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 88.1), renderer: Apple M1
W0000 00:00:1744340739.087915 7859053 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1744340739.095470 7859058 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1744340739.096763 7859046 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


6_c1.mp4


I0000 00:00:1744340804.631245 7797518 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 88.1), renderer: Apple M1
I0000 00:00:1744340804.637697 7797518 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 88.1), renderer: Apple M1
W0000 00:00:1744340804.640075 7859529 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1744340804.647019 7859529 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1744340804.649025 7859521 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


13_c3.mp4


I0000 00:00:1744340869.213517 7797518 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 88.1), renderer: Apple M1
I0000 00:00:1744340869.219835 7797518 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 88.1), renderer: Apple M1
W0000 00:00:1744340869.221702 7860105 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1744340869.228315 7860105 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1744340869.230642 7860092 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


# Timestamp 정보 추가 

In [11]:
output_dir = os.path.join(Video, "pickle")
os.makedirs(output_dir, exist_ok=True)

In [12]:
def rename_and_save_files_with_timestamp(base_path, time_1_df, time_2_df):
    first_folder = os.path.join(base_path, "1st")
    if os.path.isdir(first_folder):
        for filename in os.listdir(first_folder):
            if not filename.endswith(".csv"):
                continue
            # 예: "10_c1.csv" -> pnum="10", condition="c1"
            name_wo_ext = filename[:-4]
            parts = name_wo_ext.split("_")
            if len(parts) != 2:
                continue

            pnum_str, condition = parts
            try:
                pnum_val = int(pnum_str)
                print(pnum_val)
            except ValueError:
                continue

            # time_1_df에서 pnum == pnum_val 행 찾기
            matched = time_1_df[time_1_df["pnum"] == pnum_val]
            if matched.empty:
                # 해당 pnum이 없으면 스킵
                print('skip')
                continue
            if condition not in matched.columns:
                # c1,c2,c3 중 condition이 없으면 스킵
                print(condition, 'skip')
                continue

            # NaN 제외 -> 첫 번째 값 사용 (필요에 따라 다르게 처리 가능)
            time_val = matched[condition].iloc[0]
            # 정수 변환
            try:
                time_val = int(time_val)
            except:
                continue

            # CSV 로드
            old_path = os.path.join(first_folder, filename)
            df = pd.read_csv(old_path)

            # Timestamp 열에 time_val 더하기
            if "Timestamp" in df.columns:
                df["Timestamp"] = df["Timestamp"] + time_val

            # 새 파일명
            new_filename = f"{pnum_val}_{condition}_{time_val}.csv"
            new_path = os.path.join(output_dir, new_filename)

            # 저장
            df.to_csv(new_path, index=False)
            print(f"[1st] {filename} -> {new_filename} (Timestamp += {time_val})")

    second_folder = os.path.join(base_path, "2nd")
    if os.path.isdir(second_folder):
        for filename in os.listdir(second_folder):
            if not filename.endswith(".csv"):
                continue
            name_wo_ext = filename[:-4]
            parts = name_wo_ext.split("_")
            if len(parts) != 2:
                continue

            pnum_str, condition = parts
            # pnum 정수 변환
            try:
                pnum_val = int(pnum_str)
            except ValueError:
                continue

            matched = time_2_df[time_2_df["pnum"] == pnum_val]
            if matched.empty:
                continue

            if condition not in matched.columns:
                continue

            time_vals = matched[condition].dropna()
            if time_vals.empty:
                continue

            try:
                time_val = int(time_vals.iloc[0])
            except:
                continue

            # 2nd 폴더는 time_val에 +30
            pnum_val = pnum_val + 30

            old_path = os.path.join(second_folder, filename)
            df = pd.read_csv(old_path)

            # Timestamp 열에 time_val 더하기
            if "Timestamp" in df.columns:
                df["Timestamp"] = df["Timestamp"] + time_val

            new_filename = f"{pnum_val}_{condition}_{time_val}.csv"
            new_path = os.path.join(output_dir, new_filename)

            df.to_csv(new_path, index=False)
            print(f"[2nd] {filename} -> {new_filename} (Timestamp += {time_val})")

In [13]:
rename_and_save_files_with_timestamp(os.path.join(Video,'features'), time_1_grouped, time_2_grouped)

20
[1st] 20_c2.csv -> 20_c2_1671343685068.csv (Timestamp += 1671343685068)
3
[1st] 3_c1.csv -> 3_c1_1671078713202.csv (Timestamp += 1671078713202)
16
[1st] 16_c3.csv -> 16_c3_1671268630930.csv (Timestamp += 1671268630930)
16
[1st] 16_c2.csv -> 16_c2_1671268230560.csv (Timestamp += 1671268230560)
20
[1st] 20_c3.csv -> 20_c3_1671344101538.csv (Timestamp += 1671344101538)
20
[1st] 20_c1.csv -> 20_c1_1671343176119.csv (Timestamp += 1671343176119)
3
[1st] 3_c2.csv -> 3_c2_1671080082890.csv (Timestamp += 1671080082890)
16
[1st] 16_c1.csv -> 16_c1_1671267813860.csv (Timestamp += 1671267813860)
3
[1st] 3_c3.csv -> 3_c3_1671079535240.csv (Timestamp += 1671079535240)
22
[1st] 22_c1.csv -> 22_c1_1671353003060.csv (Timestamp += 1671353003060)
1
[1st] 1_c2.csv -> 1_c2_1671068806645.csv (Timestamp += 1671068806645)
1
[1st] 1_c3.csv -> 1_c3_1671069238335.csv (Timestamp += 1671069238335)
14
[1st] 14_c1.csv -> 14_c1_1671256497984.csv (Timestamp += 1671256497984)
18
[1st] 18_c1.csv -> 18_c1_167133211011

# 피클파일 만들기 

In [14]:
Feature = '/Users/iduli/Desktop/Ch2_25_Scientific_Data/Final/raw_Dataset/SENSORS/VIDEO/pickle'

import re 

csv_files = [f for f in os.listdir(Feature) if f.endswith('.csv')]
pattern = re.compile(r"^(\d+)_(\w+)_(\d+)\.csv$")
    
files_data = {}

In [15]:
for csv_file in csv_files:
    match = pattern.match(csv_file)
    if not match:
        continue  # 파일명이 규칙에 맞지 않으면 스킵
    
    old_pnum_str, condition, time_str = match.groups()
        
    try:
        old_pnum = int(old_pnum_str)
    except ValueError:
        continue
        
    csv_path = os.path.join(Feature, csv_file)
    df = pd.read_csv(csv_path)
        
    if old_pnum not in files_data:
        files_data[old_pnum] = {}

    files_data[old_pnum][condition] = df

# 3) old_pnum들을 정렬하여, 새 pnum(1부터 1씩 증가)로 매핑
sorted_pnums = sorted(files_data.keys())
    
old_to_new = {}
new_pnum = 1
for p in sorted_pnums:
    old_to_new[p] = new_pnum
    print(p, new_pnum)
    new_pnum += 1


1 1
2 2
3 3
4 4
8 5
10 6
12 7
13 8
14 9
16 10
18 11
19 12
20 13
21 14
22 15
23 16
25 17
26 18
27 19
31 20
32 21
34 22
35 23
36 24
38 25
40 26
41 27
42 28
43 29
44 30
45 31


In [16]:
final_dict = {}
for old_pnum, cond_dict in files_data.items():
    npnum = old_to_new[old_pnum]  # 새 pnum
    final_dict[npnum] = {}       # 서브 dict
    
    for cond, df in cond_dict.items():
        final_dict[npnum][cond] = df#
#check data

In [17]:
# final_dict의 메인 키들을 정렬한 뒤, 각 값(서브 딕셔너리)의 키들도 "c1", "c2", "c3" 순으로 정렬합니다.
sorted_final = {}
for main_key in sorted(final_dict.keys()):
    sub_dict = final_dict[main_key]
    sorted_sub = {}
    for key in ["c1", "c2", "c3"]:
        if key in sub_dict:
            sorted_sub[key] = sub_dict[key]
    sorted_final[main_key] = sorted_sub

print(sorted_final[20]['c2'])

         Timestamp  rotation_x  rotation_y  rotation_z  Total Movement  \
0    1708134415179           1          -0           0             644   
1    1708134415179           1          -0           0             648   
2    1708134415212           1          -0           0             646   
3    1708134415246           1          -0           0             645   
4    1708134415279           1          -0           0             648   
...            ...         ...         ...         ...             ...   
7779 1708134674446           1           0           0             615   
7780 1708134674479           1           0           0             619   
7781 1708134674512           1           0           0             621   
7782 1708134674546           1           0           0             626   
7783 1708134674579           1           0           0             629   

      Gaze X  Gaze Y  Left Lip Movement  Right Lip Movement  Blink Count  \
0          0      -1               

In [18]:
destination = '/Users/iduli/Desktop/Ch2_25_Scientific_Data/Final/Dataset_publish/K-EmoWorker/SENSORS'

with open(os.path.join(destination,'video.pkl'),'wb') as f:
    pickle.dump(sorted_final, f)